In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression,LogisticRegressionCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from imblearn.pipeline import make_pipeline as make_pipeline2 
from sklearn.metrics import make_scorer,f1_score

In [2]:
train = pd.read_csv('MetadataTrainModified.csv')
train.head()

,ID,CLASS,SEX,AGE,POSITION,MinL,MaxL,AvgL,VarL,MinH,...,DiffH,DiffL,Area,Perimeter,Aspect Ratio,Distance,Compactness,Roundness,Convexity,Solidity
0,ISIC_0000000,2,1,0.647059,0.00,0.109091,0.878205,0.565203,0.417285,0.0,...,0.024649,0.301533,0.475569,0.295352,0.693396,0.134803,0.829122,0.873353,0.749499,0.972363
1,ISIC_0000001,2,1,0.352941,0.00,0.072727,0.865385,0.636809,0.083832,0.0,...,0.672806,0.657611,0.071520,0.134933,0.851351,0.142167,0.597424,0.861772,0.640474,0.904813
2,ISIC_0000002,1,1,0.705882,0.71,0.000000,0.814103,0.617074,0.177336,0.0,...,0.427647,0.502230,0.200806,0.202149,0.996241,0.106458,0.747342,0.754826,0.765408,0.847599
3,ISIC_0000003,2,0,0.352941,0.71,0.206061,0.961538,0.703693,0.301640,0.0,...,0.277054,0.463123,0.341318,0.259620,0.916667,0.175795,0.770138,0.895547,0.713340,0.942955
4,ISIC_0000004,1,0,0.941176,0.29,0.000000,0.974359,0.228720,0.660375,0.0,...,0.195897,0.121372,0.349434,0.217391,0.814590,0.071795,1.000000,0.932791,0.844593,0.987723


In [3]:
test = pd.read_csv('MetadataTestModified.csv')
test.head()

,ID,SEX,AGE,POSITION,MinL,MaxL,AvgL,VarL,MinH,MaxH,...,DiffH,DiffL,Area,Perimeter,Aspect Ratio,Distance,Compactness,Roundness,Convexity,Solidity
0,ISIC_0000006,1,0.294118,0.29,0.000000,0.754839,0.587001,0.103270,0.0,0.988372,...,0.538090,0.584072,0.095695,0.124134,0.832370,0.115471,0.490080,0.825862,0.592566,0.892031
1,ISIC_0000011,1,0.411765,0.86,0.000000,0.787097,0.532010,0.238288,0.0,0.994186,...,0.505371,0.449342,0.187519,0.144732,0.826772,0.137741,0.706445,0.868107,0.693920,0.920792
2,ISIC_0000014,0,0.411765,0.29,0.000000,0.800000,0.655311,0.067169,0.0,0.994186,...,0.494732,0.398469,0.291932,0.197594,0.743363,0.082348,0.590062,0.829090,0.648940,0.940831
3,ISIC_0000018,0,0.352941,0.29,0.363095,0.600000,0.641963,0.064451,0.0,0.994186,...,0.576864,0.464251,0.245173,0.199234,0.651652,0.110793,0.487424,0.804434,0.598776,0.918776
4,ISIC_0000022,1,0.647059,0.86,0.238095,0.535484,0.559884,0.075246,0.0,0.994186,...,0.415722,0.202708,0.425469,0.187204,0.680590,0.151280,0.958078,0.873483,0.805619,0.955092


In [4]:
#ATTENTION ENLEVER CETTE CELLULE AVANT DE RENDRE LE NOTEBOOK

gt = pd.read_csv('./arnaque/arnaque.csv').to_numpy()[:,1]
print(gt)
weights = [0.7005531, 0.24592265, 0.95261733, 3.64804147, 1.20674543, 13.19375, 12.56547619, 5.04219745]

[2 2 2 ... 3 1 5]


In [5]:
def weighted_accuracy(pred,w):
    wacc = 0
    for i in range(len(pred)):
        label = gt[i]
        if pred[i]==label:
            wacc += w[label-1]
    return wacc/len(pred)


In [6]:
t = train.to_numpy()
print(t.shape)
t2 = test.to_numpy()
print(t2.shape)

(18998, 26)
(6333, 25)


In [7]:
train_labels = np.array(t[:,1]).astype(int)
#print(train_labels)
train_data = t[:,2:]
#print(train_data)
test_data = t2[:,1:]
#print(test_data)

In [8]:
train_lab = train_labels.reshape(-1,1)
ohe = OneHotEncoder()
ohe.fit(train_lab)
labels = ohe.transform(train_lab).toarray()
#print(labels.shape)
#print(labels)

# Linear

In [10]:
reg = LinearRegression()
reg.fit(train_data,labels)
print(reg.score(train_data,labels))

0.11096514268033889


In [11]:
test_labels = reg.predict(test_data)
test_labels = np.argmax(test_labels,axis=1)+1
print(test_labels)

[2 2 2 ... 3 1 2]


In [12]:
print('Weighted accuracy:',weighted_accuracy(test_labels,weights))

Weighted accuracy: 0.21904128681983634


# LogitCV

In [132]:
logitcv= LogisticRegressionCV(Cs=[0.5,0.75,1],cv=10,penalty='l2',class_weight='balanced',solver='saga',max_iter=1000 ,multi_class='multinomial')
logitcv.fit(train_data,train_labels)
print(logitcv.score(train_data,train_labels))
print(logitcv.C_)

c:\Users\maelm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\maelm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\maelm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\maelm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\maelm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the 

0.1498578797768186
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]


c:\Users\maelm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [13]:
logit = LogisticRegression(penalty='l2',C=0.5,class_weight='balanced',solver='saga',max_iter=10000 ,multi_class='multinomial')
logit.fit(train_data,train_labels)
print(logit.score(train_data,train_labels))

0.3725655332140225


c:\Users\maelm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [14]:
test_labels = logit.predict(test_data)
print(test_labels)
print('Weighted accuracy:',weighted_accuracy(test_labels,weights))

[2 1 2 ... 4 1 6]
Weighted accuracy: 0.3492606184004439


# Trees

In [16]:
tree = DecisionTreeClassifier(criterion='entropy',random_state=23)
tree.fit(train_data,labels)
print(tree.score(train_data,labels))
test_labels = tree.predict(test_data)
test_labels = np.argmax(test_labels,axis=1)+1
print('Weighted accuracy:',weighted_accuracy(test_labels,weights))

1.0
Weighted accuracy: 0.29412097254382125


In [77]:
RF =RandomForestClassifier(criterion='entropy',random_state=23)
p_grid_RF = {'n_estimators': [10,15,20,25,30], 'min_samples_leaf': [2,3,4,5,6], 'max_features': ['sqrt','log2']}   

grid_RF = GridSearchCV(estimator=RF, param_grid=p_grid_RF, scoring='balanced_accuracy', cv=5,verbose=2)
grid_RF.fit(train_data,labels)
print(grid_RF.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=10; total time=   2.1s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=10; total time=   2.0s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=10; total time=   2.0s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=10; total time=   2.0s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=10; total time=   2.2s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=15; total time=   3.3s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=15; total time=   3.1s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=15; total time=   3.1s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=15; total time=   3.2s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=15; total time=   3.4s
[CV] END max_features=sqrt, min_samples_leaf=2, n_estimators=20; total time=   4.1s
[CV] END max_f

In [78]:
print(grid_RF.best_params_)

{'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 30}


In [17]:
tree = RandomForestClassifier(n_estimators=100,criterion='entropy',max_features='sqrt',random_state=23)
tree.fit(train_data,labels)
print(tree.score(train_data,labels))
test_labels = tree.predict(test_data)
test_labels = np.argmax(test_labels,axis=1)+1
print('Weighted accuracy:',weighted_accuracy(test_labels,weights))

0.9999473628803032
Weighted accuracy: 0.22261793693352455


# SVM

In [30]:
svm = SVC(verbose=True,class_weight='balanced',random_state=23)
svm.fit(train_data,train_labels)
print(svm.score(train_data,train_labels))
test_labels = svm.predict(test_data)
print('Weighted accuracy:',weighted_accuracy(test_labels,weights))

[LibSVM]0.47494473102431833
Weighted accuracy: 0.4329789831280619


In [36]:
p_grid={'C':[10,15,20],'gamma':[0.1,0.2,0.5,1]}
grid_svm = GridSearchCV(SVC(),p_grid,scoring='balanced_accuracy',verbose=3)
grid_svm.fit(train_data,train_labels)

Fitting 5 folds for each of 55 candidates, totalling 275 fits
[CV 1/5] END ...................C=20, gamma=0.5;, score=0.189 total time=  56.4s
[CV 2/5] END ...................C=20, gamma=0.5;, score=0.266 total time=  34.7s
[CV 3/5] END ...................C=20, gamma=0.5;, score=0.297 total time=  34.5s
[CV 4/5] END ...................C=20, gamma=0.5;, score=0.302 total time=  33.0s
[CV 5/5] END ...................C=20, gamma=0.5;, score=0.294 total time=  33.3s
[CV 1/5] END ...................C=20, gamma=0.8;, score=0.192 total time=  34.2s
[CV 2/5] END ...................C=20, gamma=0.8;, score=0.295 total time=  40.3s
[CV 3/5] END ...................C=20, gamma=0.8;, score=0.330 total time=  41.7s
[CV 4/5] END ...................C=20, gamma=0.8;, score=0.346 total time=  36.2s
[CV 5/5] END ...................C=20, gamma=0.8;, score=0.335 total time=  35.8s
[CV 1/5] END .....................C=20, gamma=1;, score=0.191 total time=  39.2s
[CV 2/5] END .....................C=20, gamma=1

In [38]:
print(grid_svm.best_params_)

{'C': 30, 'gamma': 1.5}


In [40]:
svm = SVC(verbose=True,C=20,gamma=1,class_weight='balanced')
svm.fit(train_data,train_labels)
print(svm.score(train_data,train_labels))
test_labels = svm.predict(test_data)
print('Weighted accuracy:',weighted_accuracy(test_labels,weights))

[LibSVM]0.7828192441309612
Weighted accuracy: 0.4625326763571815
